In [2]:
import networkx as nx
import pandas as pd
from collections import Counter, defaultdict
from tqdm import tqdm
import os
import pickle
import hashlib

#### LOAD IN BIG GRAPH

In [ ]:
G = nx.read_graphml(f"procurement_graph_none.graphml")

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x1029f1c40>>
Traceback (most recent call last):
  File "/Users/wiktorrajca/Library/Python/3.9/lib/python/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [5]:
len(G.nodes)
len(G.edges)

7880318

#### SPLIT BIG GRAPH BY COUNTRY

In [2]:
def split_graph_by_country(G, output_dir="graphs_by_country_optimized"):
    import os
    from tqdm import tqdm
    os.makedirs(output_dir, exist_ok=True)

    # Step 1: Group winner nodes by country
    country_winners = {}
    for node, data in tqdm(G.nodes(data=True), desc="Grouping nodes by country"):
        if data.get("type") == "Company" and data.get("bid_winner"):
            country = str(data.get("win_country_code", "Unknown"))[:2]
            country_winners.setdefault(country, []).append(node)

    # Step 2: Build a neighbor map only once
    print("⚙️  Indexing neighbors...")
    neighbor_map = {}
    for node in tqdm(G.nodes(), desc="Indexing neighbors"):
        neighbors = set(G.successors(node)).union(G.predecessors(node))
        neighbor_map[node] = neighbors

    # Step 3: Build subgraphs efficiently using neighbor map
    split_graphs = {}
    for country, nodes in tqdm(country_winners.items(), desc="Creating subgraphs"):
        sub_nodes = set(nodes)  # start with winner nodes
        for node in nodes:
            sub_nodes.update(neighbor_map.get(node, set()))  # add all neighbors

        subgraph = G.subgraph(sub_nodes).copy()
        split_graphs[country] = subgraph

        path = os.path.join(output_dir, f"graph_{country}.graphml")
        nx.write_graphml(subgraph, path)
        tqdm.write(f"✅ Saved {country} ({len(subgraph.nodes)} nodes) → {path}")

    return split_graphs

In [ ]:
split_graph_by_country(G)

Grouping nodes by country: 100%|██████████| 11524092/11524092 [01:07<00:00, 170283.96it/s]


⚙️  Indexing neighbors...


Creating subgraphs:   1%|          | 1/175 [22:27<65:08:53, 1347.90s/it]

✅ Saved fr (2469037 nodes) → graphs_by_country_optimized/graph_fr.graphml


Creating subgraphs:   1%|          | 2/175 [27:36<35:23:47, 736.57s/it] 

...

✅ Saved cn (3404 nodes) → graphs_by_country_optimized/graph_cn.graphml


Creating subgraphs:  11%|█▏        | 20/175 [1:24:27<1:09:37, 26.95s/it]

✅ Saved pt (60978 nodes) → graphs_by_country_optimized/graph_pt.graphml


Creating subgraphs:  12%|█▏        | 21/175 [1:24:43<1:01:24, 23.92s/it]

✅ Saved ie (151803 nodes) → graphs_by_country_optimized/graph_ie.graphml


In [3]:
Gf = nx.read_graphml(f"graphs_by_country_optimized/graph_fr.graphml")

In [4]:
print("Nodes ",len(Gf.nodes))
print("Edges ", len(Gf.edges))

Nodes  2469037
Edges  1651874


#### SUMMARIZE GRAPH

In [38]:
def summarize_graph(G):
    print("\n===== Graph Summary =====")
    print(f"Total Nodes: {len(G.nodes)}")
    print(f"Total Edges: {len(G.edges)}")

    node_types = [G.nodes[n].get("type", "Unknown") for n in G.nodes]
    node_type_counts = Counter(node_types)
    print("\nNode Types:")
    for node_type, count in node_type_counts.items():
        print(f"  - {node_type}: {count}")

    edge_relationships = [G.edges[e].get("relationship", "Unknown") for e in G.edges]
    edge_type_counts = Counter(edge_relationships)
    print("\nEdge Types:")
    for rel, count in edge_type_counts.items():
        print(f"  - {rel}: {count}")

    flagged_count = sum(1 for n in G.nodes if G.nodes[n].get("flagged") == 'True')
    print(f"\nFlagged Companies: {flagged_count}")
    print("=========================\n")

In [6]:
summarize_graph(Gf)


===== Graph Summary =====
Total Nodes: 2469037
Total Edges: 1651874

Node Types:
  - Company: 1150169
  - Procurement: 1318863
  - Flagged: 5

Edge Types:
  - WON: 1318863
  - CONTROLS: 34460
  - OWNS: 298340
  - SUBSIDIARY_OF: 122
  - FLAGGED_LINK: 89

Flagged Companies: 14078



In [36]:
def summarize_flagged_entity_relations_separately(G, top_k=10):
    print("===== FLAGGED ENTITY RELATION SUMMARY (by type) =====")

    flagged_companies = {}
    flagged_people = {}

    for node, data in tqdm(G.nodes(data=True), desc="Scanning nodes", unit="node"):
        if data.get("flagged") == "True" and data.get("type") == "Company":
            flagged_companies[node] = data.get("name", "Unknown")
        elif data.get("type") == "Flagged":
            flagged_people[node] = data.get("name", "Unknown")

    won_counts = defaultdict(int)
    owns_counts = defaultdict(int)
    controls_counts = defaultdict(int)
    subsidiary_counts = defaultdict(int)
    flagged_links = defaultdict(set)

    for u, v, d in tqdm(G.edges(data=True), desc="Scanning edges", unit="edge"):
        rel = d.get("relationship")
        if rel == "WON" and u in flagged_companies:
            won_counts[u] += 1
        if rel == "OWNS" and u in flagged_companies:
            owns_counts[u] += 1
        if rel == "CONTROLS" and u in flagged_companies:
            controls_counts[u] += 1
        if rel == "SUBSIDIARY_OF" and u in flagged_companies:
            subsidiary_counts[u] += 1
        if rel == "FLAGGED_LINK" and u in flagged_people:
            flagged_links[u].add(v)

    def print_top(mapping, label, name_map):
        print(f"\nTop {top_k} by {label}:")
        top = sorted(mapping.items(), key=lambda x: x[1] if isinstance(x[1], int) else len(x[1]), reverse=True)[:top_k]
        df = pd.DataFrame([
            {
                "Name": name_map.get(k, "Unknown"),
                "ID": k,
                label: v if isinstance(v, int) else len(v)
            }
            for k, v in top
        ])
        print(df.to_string(index=False))

    print(f"\nFlagged Companies Total: {len(flagged_companies)}")
    print(f"Flagged People Total: {len(flagged_people)}")

    print(f"\nFlagged Companies with WON edges: {len(won_counts)}")
    print(f"Flagged Companies with OWNS edges: {len(owns_counts)}")
    print(f"Flagged Companies with CONTROLS edges: {len(controls_counts)}")
    print(f"Flagged Companies with SUBSIDIARY_OF edges: {len(subsidiary_counts)}")
    print(f"Flagged People with FLAGGED_LINK connections: {len(flagged_links)}")

    print_top(won_counts, "Contracts Won", flagged_companies)
    print_top(owns_counts, "Companies Owned", flagged_companies)
    print_top(controls_counts, "Companies Controlled", flagged_companies)
    print_top(subsidiary_counts, "Subsidiary Links", flagged_companies)
    print_top(flagged_links, "Linked Companies", flagged_people)

    print("\n===== END OF SUMMARY =====\n")

In [64]:
sanc = pd.read_csv("/Users/wiktorrajca/Desktop/Research/URAP_Fedyk/data/Black_Lists/Black_Lists_Data_Sets/open_sanctions.csv")
count_contains = sanc['name'].str.contains('SDS', na=False).sum()
count_exact = (sanc['name'] == 'SDS').sum()
print(count_contains, count_exact)

8 1


/var/folders/r2/9vnxdxyd26380tms_8k538b40000gn/T/ipykernel_1517/3005866531.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  sanc = pd.read_csv("/Users/wiktorrajca/Desktop/Research/URAP_Fedyk/data/Black_Lists/Black_Lists_Data_Sets/open_sanctions.csv")


In [75]:
import pandas as pd
from collections import defaultdict
import re

def summarize_flagged_entities_to_csv(G, output_path="flagged_entity_summary.csv"):
    """
    Summarizes flagged companies and people along with their graph relationships (WON, OWNS, CONTROLS, etc.).
    Outputs the summary as a CSV.
    
    Parameters:
        G (networkx.Graph): The input graph.
        output_path (str): File path to save the summary CSV.
    
    Returns:
        pd.DataFrame: The summary DataFrame.
    """
    flagged_summary = []

    for node, data in G.nodes(data=True):
        if data.get("flagged") == "True" or data.get("type") == "Flagged":
            node_id = node
            node_type = data.get("type", "Unknown")
            name = data.get("name", "Unknown")

            # Try to extract the sanction_id if present in flagged_reason
            flagged_reason = data.get("flagged_reason", "")
            match = re.search(r"Matched to flagged entity (\S+)", flagged_reason)
            sanction_id = match.group(1) if match else ""

            # Count relationships from this node
            won_count = 0
            owns_count = 0
            controls_count = 0
            subsidiary_count = 0
            flagged_links = 0

            for _, tgt, edata in G.out_edges(node, data=True):
                rel = edata.get("relationship")
                if rel == "WON":
                    won_count += 1
                elif rel == "OWNS":
                    owns_count += 1
                elif rel == "CONTROLS":
                    controls_count += 1
                elif rel == "SUBSIDIARY_OF":
                    subsidiary_count += 1
                elif rel == "FLAGGED_LINK":
                    flagged_links += 1

            for src, _, edata in G.in_edges(node, data=True):
                rel = edata.get("relationship")
                if rel == "FLAGGED_LINK":
                    flagged_links += 1

            flagged_summary.append({
                "ID": node_id,
                "Sanction_ID": sanction_id,
                "Name": name,
                "Type": node_type,
                "Contracts_Won": won_count,
                "Owns_Companies": owns_count,
                "Controls_Companies": controls_count,
                "Subsidiary_Links": subsidiary_count,
                "Flagged_Links": flagged_links
            })

    df = pd.DataFrame(flagged_summary)
    df.to_csv(output_path, index=False)
    print(f"✅ Summary saved to {output_path}")
    return df

In [12]:
df = summarize_flagged_entities_to_csv(Gf)

✅ Summary saved to flagged_entity_summary.csv


In [13]:
df

,ID,Sanction_ID,Name,Type,Contracts_Won,Owns_Companies,Controls_Companies,Subsidiary_Links,Flagged_Links
0,RU79752370,usgsa-s4mr55c8r,Smart,Company,1,0,0,0,0
1,RU40924122N,NK-4ZURfkkRkubWm2FfdVqq3F,Altair,Company,2,0,0,0,0
2,RU36275132,NK-4ZURfkkRkubWm2FfdVqq3F,Altair,Company,2,1,0,0,0
3,RU34543568N,NK-Gbik6Hp3G8o9nPkMzyq68M,Antares,Company,1,0,0,0,0
4,RU05255392,NK-Gbik6Hp3G8o9nPkMzyq68M,Antares,Company,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...
14078,US126303612L,usgsa-s4mr3pyw9,PCI Corp,Company,1,0,0,0,0
14079,US131748537L,usgsa-s4mr3pyw9,PCI,Company,1,0,0,0,0
14080,RU44258076,usgsa-s4mr3p1vn,SDS,Company,3,0,0,0,0
14081,US257613749L,usgsa-s4mr3pr2j,"M.I.R.A., Inc.",Company,1,0,0,0,0


#### LOOKING INTO GRAPH (ONE TO MANY PROBLEM)

In [14]:
def group_flagged_summary_by_name(df):
    grouped = df.groupby("Name").agg({
        "ID": pd.Series.nunique,
        "Sanction_ID": pd.Series.nunique,  # Keep the first seen sanction ID
        "Type": pd.Series.nunique,         # Keep the first seen type
        "Contracts_Won": "sum",
        "Owns_Companies": "sum",
        "Controls_Companies": "sum",
        "Subsidiary_Links": "sum",
        "Flagged_Links": "sum"
    }).reset_index()

    return grouped

In [15]:
grouped = group_flagged_summary_by_name(df)
grouped

,Name,ID,Sanction_ID,Type,Contracts_Won,Owns_Companies,Controls_Companies,Subsidiary_Links,Flagged_Links
0,( Antares CO Ltd),1,1,1,1,0,0,0,0
1,(E)Motion,1,1,1,2,0,0,0,0
2,<<Altair>>,1,1,1,2,0,0,0,0
3,A-Mig+,1,1,1,1,0,0,0,0
4,A-N-O,1,1,1,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...
761,htc,1,1,1,2,0,0,0,0
762,mira,5,1,1,5,2,0,0,0
763,sds,2,1,1,6,0,0,0,0
764,smART,1,1,1,1,0,0,0,0


In [16]:
grouped[grouped['ID']>grouped['Sanction_ID']].sort_values('ID')

,Name,ID,Sanction_ID,Type,Contracts_Won,Owns_Companies,Controls_Companies,Subsidiary_Links,Flagged_Links
765,smart,2,1,1,2,0,0,0,0
587,SRA & CO,2,1,1,6,0,0,0,0
180,DME Company LLC,2,1,1,6,0,0,0,0
592,SRA Corp,2,1,1,6,0,0,0,0
435,M.O.T INC,2,1,1,2,2,0,0,0
...,...,...,...,...,...,...,...,...,...
294,Gross,369,1,1,369,91,1,0,0
457,Mira,634,1,1,634,301,3,0,0
102,Antares,2321,1,1,2321,645,22,0,0
636,Smart,3216,1,1,3216,403,6,0,0


In [17]:
def group_orbis_entities_by_name(G):
    """
    Groups Orbis company nodes in the graph by their 'name' attribute.
    
    Returns:
        dict: name → list of node IDs with that name
    """
    from collections import defaultdict

    name_to_ids = defaultdict(list)

    for node, data in G.nodes(data=True):
        if data.get("type") == "Company":
            name = data.get("name", "").strip()
            if name:
                name_to_ids[name].append(node)

    return dict(name_to_ids)

In [18]:
def map_sanction_ids_to_orbis_ids(G):
    """
    Maps each unique sanction entity ID to a list of matched Orbis company IDs.
    Only considers LegalEntities with a 'flagged_reason' field in the format:
    'Matched to flagged entity <ID>'
    
    Returns:
        dict: sanction_id → list of company node IDs
    """
    import re
    from collections import defaultdict

    sanction_map = defaultdict(list)

    for node, data in G.nodes(data=True):
        if data.get("type") == "Company" and data.get("flagged") == "True":
            reason = data.get("flagged_reason", "")
            match = re.search(r"Matched to flagged entity (\S+)", reason)
            if match:
                sanction_id = match.group(1)
                sanction_map[sanction_id].append(node)

    return dict(sanction_map)

In [19]:
group_orbis_entities_by_name(Gf)['Altair']

['RU14282769',
 'RU19106600',
 'RU40924122N',
 'RU36275132',
 'RU85542139',
 'RU78403176',
 'RU67972474',
 'RU20034231',
 'RU73515367',
 'RU26529382',
 'RU69001148',
 'RU24147110',
 'RU89049938',
 'RU24712461N',
 'RU89022785',
 'RU23054493N',
 'RU72856511N',
 'RU54225847N',
 'RU41791396N',
 'RU71483381',
 'RU21803811',
 'RU04416186',
 'RU95206106',
 'RU03935777',
 'RU84209946',
 'RU45198436N',
 'RU48935786',
 'RU90627270',
 'RU81514873',
 'RU06601309',
 'RU33659180',
 'RU63348678',
 'RU54178258',
 'RU53724264N',
 'RU17298794N',
 'RU20041705',
 'RU40567142',
 'RU98256754',
 'RU94100268',
 'RU10627065N',
 'RU61083856',
 'RU46275009',
 'RU97234609',
 'RU39338985N',
 'RU39058530N',
 'RU54309805',
 'RU77386234',
 'RU70347892',
 'RU44551155',
 'RU54283253',
 'RU29165176',
 'RU48962814',
 'RU92119772',
 'RU75118044',
 'RU91609959',
 'RU19330326',
 'RU16002751',
 'RU50455645',
 'RU69379674',
 'US263513039L',
 'RU44310919',
 'RU05058932',
 'RU82504167',
 'RU09910465',
 'RU71050467',
 'RU7308070

#### GRAPH PRUNING

In [21]:
def clean_node_attributes(G):
    """
    Ensures all node attributes are stored as strings and missing values are handled.
    """
    for node in G.nodes:
        for attr, value in G.nodes[node].items():
            if pd.isna(value) or value is None:  # Replace NaN/None with 'Unknown'
                G.nodes[node][attr] = "Unknown"
            else:
                G.nodes[node][attr] = str(value)  # Convert everything to a string

def save_graph(G, country_code):
    """
    Saves the graph to GraphML format.
    """
    clean_node_attributes(G)  # Ensure consistent types before saving
    nx.write_graphml(G, f"procurement_graph_{country_code}_clean.graphml")
    print(f"✅ Graph saved to procurement_graph_{country_code}_clean.graphml")

In [22]:
import networkx as nx
from collections import defaultdict, deque
import copy

def label_and_clean_matched_companies_create_new(G):
    """
    Creates a cleaned copy of the graph with company nodes labeled by match confidence,
    removes 'SuperLow' confidence matches, and recursively removes disconnected Orbis nodes.
    """
    G_cleaned = copy.deepcopy(G)

    # Step 1: Group company nodes by name
    name_to_nodes = defaultdict(list)
    name_to_unique_ids = defaultdict(set)

    for node, data in G.nodes(data=True):
        if data.get("type") == "Company" and data.get("bid_winner") == "True":
            name = data.get("name")
            national_id = data.get("win_nationalid")
            name_to_nodes[name].append(node)
            if national_id:
                name_to_unique_ids[name].add(national_id)

    # Step 2: Label confidence
    last_win_country = None
    superlow_nodes = set()
    for name, nodes in name_to_nodes.items():
        expected_matches = len(name_to_unique_ids[name])
        for node in nodes:
            data = G_cleaned.nodes[node]
            bvd_country = str(data.get("bvdidnumber", ""))[:2].upper()
            win_country = str(data.get("win_country_code", ""))[:2].upper()
            last_win_country = win_country

            country_match = bvd_country == win_country
            total_matches = len(nodes)

            # Determine confidence
            if country_match and total_matches == 1:
                confidence = "Very High"
            elif country_match and total_matches <= expected_matches:
                confidence = "High"
            elif not country_match and total_matches <= expected_matches:
                confidence = "Medium"
            elif country_match and total_matches >= expected_matches:
                confidence = "Low"
            else:
                confidence = "SuperLow"

            data["match_confidence"] = confidence

            if confidence == "SuperLow":
                superlow_nodes.add(node)

    # Step 3: Remove all SuperLow nodes and clean neighbors
    print(f"🧹 Removing {len(superlow_nodes)} SuperLow nodes and cleaning neighbors...")
    G_cleaned.remove_nodes_from(superlow_nodes)

    # Step 4: Keep only nodes reachable from procurement
    reachable = set()
    for node, data in G_cleaned.nodes(data=True):
        if data.get("type") == "Procurement":
            queue = deque([node])
            visited = set()
            while queue:
                current = queue.popleft()
                if current in visited:
                    continue
                visited.add(current)
                reachable.add(current)
                neighbors = set(G_cleaned.successors(current)).union(G_cleaned.predecessors(current))
                queue.extend(neighbors - visited)

    all_nodes = set(G_cleaned.nodes())
    unreachable_nodes = all_nodes - reachable
    G_cleaned.remove_nodes_from(unreachable_nodes)

    # Step 5: Remove final orphans
    orphans = [n for n in G_cleaned.nodes if G_cleaned.degree(n) == 0]
    G_cleaned.remove_nodes_from(orphans)

    save_graph(G_cleaned, last_win_country)
    print(f"✅ Cleaned graph created. Removed {len(superlow_nodes)} SuperLow nodes and {len(unreachable_nodes)} unreachable nodes.")
    return G_cleaned

In [23]:
Gf_clean = label_and_clean_matched_companies_create_new(Gf)

🧹 Removing 697485 SuperLow nodes and cleaning neighbors...
✅ Graph saved to procurement_graph_FR_clean.graphml
✅ Cleaned graph created. Removed 697485 SuperLow nodes and 116091 unreachable nodes.


#### INTEGRITY

In [24]:
def check_graph_integrity(G):
    """
    Checks if the graph meets integrity expectations:
    - No orphan nodes (nodes with no edges)
    - Only isolated pairs are connected by 'WON' edges
    - Each procurement node has only one incoming 'WON' edge
    """
    print("🧪 Running graph integrity checks...")

    # 1. Orphan nodes
    orphan_nodes = [n for n in G.nodes if G.degree(n) == 0]
    if orphan_nodes:
        print(f"❌ Found {len(orphan_nodes)} orphan nodes.")
    else:
        print("✅ No orphan nodes found.")

    # 2. Isolated pairs must be connected only by 'WON' edges
    only_one_edge = []
    for u, v in G.edges:
        if G.degree(u) == 1 and G.degree(v) == 1:
            rel = G.edges[u, v].get("relationship")
            if rel != "WON":
                only_one_edge.append((u, v, rel))

    if only_one_edge:
        print(f"❌ Found {len(only_one_edge)} node pairs connected only by non-WON edges.")
        for u, v, rel in only_one_edge[:5]:  # Show a few examples
            print(f"   - ({u}, {v}) via {rel}")
    else:
        print("✅ All isolated pairs are connected by 'WON' edges only.")

    # 3. Each Procurement node should have at most one incoming 'WON' edge
    bad_procurements = []
    for node, data in G.nodes(data=True):
        if data.get("type") == "Procurement":
            won_in_edges = [
                (u, v) for u, v in G.in_edges(node)
                if G.edges[u, v].get("relationship") == "WON"
            ]
            if len(won_in_edges) > 1:
                bad_procurements.append((node, len(won_in_edges)))

    if bad_procurements:
        print(f"❌ Found {len(bad_procurements)} procurement nodes with multiple 'WON' edges.")
        for pid, count in bad_procurements[:5]:
            print(f"   - Procurement {pid} has {count} winners.")
    else:
        print("✅ All procurements have at most one winning company.")

    print("🧪 Integrity check complete.\n")

In [25]:
check_graph_integrity(Gf_clean)

🧪 Running graph integrity checks...
✅ No orphan nodes found.
✅ All isolated pairs are connected by 'WON' edges only.
✅ All procurements have at most one winning company.
🧪 Integrity check complete.



In [26]:
summarize_graph(Gf)


===== Graph Summary =====
Total Nodes: 2469037
Total Edges: 1651874

Node Types:
  - Company: 1150169
  - Procurement: 1318863
  - Flagged: 5

Edge Types:
  - WON: 1318863
  - CONTROLS: 34460
  - OWNS: 298340
  - SUBSIDIARY_OF: 122
  - FLAGGED_LINK: 89

Flagged Companies: 14078



In [27]:
summarize_graph(Gf_clean)


===== Graph Summary =====
Total Nodes: 606730
Total Edges: 431038

Node Types:
  - Procurement: 270132
  - Company: 336594
  - Flagged: 4

Edge Types:
  - WON: 270132
  - CONTROLS: 12973
  - OWNS: 147819
  - FLAGGED_LINK: 37
  - SUBSIDIARY_OF: 77

Flagged Companies: 659



In [28]:
summarize_flagged_entity_relations_separately(Gf_clean)

===== FLAGGED ENTITY RELATION SUMMARY (by type) =====


Scanning edges: 100%|██████████| 431038/431038 [00:00<00:00, 1172791.11edge/s]


Flagged Companies Total: 659
Flagged People Total: 4

Flagged Companies with WON edges: 644
Flagged Companies with OWNS edges: 201
Flagged Companies with CONTROLS edges: 27
Flagged Companies with SUBSIDIARY_OF edges: 0
Flagged People with FLAGGED_LINK connections: 4

Top 10 by Contracts Won:
               Name              ID  Contracts Won
            SAS SRA     FR839969888              3
             S.R.A.     FR525157004              3
SDS Company Co.,Ltd KR1357110170177              3
                SDS     FR444819726              3
                SDS     FR801130105              3
       D-M-E CO INC    US191783640L              3
              S.D.S     FR432553139              3
           DME Llc,    US317049218L              3
        SDS LTD INC    US135683443L              3
           SDS Co.,    US316895450L              3

Top 10 by Companies Owned:
             Name              ID  Companies Owned
            Smart     FR848990743               18
           Alta

In [29]:
summarize_flagged_entity_relations_separately(Gf)

===== FLAGGED ENTITY RELATION SUMMARY (by type) =====


Scanning edges: 100%|██████████| 1651874/1651874 [00:01<00:00, 1134385.37edge/s]


Flagged Companies Total: 14078
Flagged People Total: 5

Flagged Companies with WON edges: 14032
Flagged Companies with OWNS edges: 2440
Flagged Companies with CONTROLS edges: 152
Flagged Companies with SUBSIDIARY_OF edges: 0
Flagged People with FLAGGED_LINK connections: 5

Top 10 by Contracts Won:
       Name              ID  Contracts Won
        SDS      RU33409052              3
        SDS      NL57100411              3
        SDS    US149304995L              3
        SDS    US127065902L              3
     S.D.S.    BE2065908374              3
        DME      RU33970469              3
        SDS     RU43732104N              3
SDS Co.,Ltd KR1201110656720              3
        SDS      RU50909810              3
    SDS LLC    US266102738L              3

Top 10 by Companies Owned:
   Name              ID  Companies Owned
   Mira     BG836144081               22
 Altair      RU05368064               20
  Smart     FR848990743               18
 Altair   IT02301900060            

#### WORK ON SANCTIONS

In [30]:
df = summarize_flagged_entities_to_csv(Gf_clean)

✅ Summary saved to flagged_entity_summary.csv


In [31]:
grouped = group_flagged_summary_by_name(df)

In [32]:
grouped[grouped['ID']>grouped['Sanction_ID']].sort_values('ID')

,Name,ID,Sanction_ID,Type,Contracts_Won,Owns_Companies,Controls_Companies,Subsidiary_Links,Flagged_Links
263,Novacom,2,1,1,2,1,0,0,0
322,SRA,2,1,1,6,1,0,0,0
212,Interconsult,2,1,1,2,0,0,0,0
392,Symed,2,1,1,2,4,1,0,0
287,S.D.S,2,1,1,6,0,0,0,0
141,Erys,2,1,1,2,0,0,0,0
288,S.D.S.,2,1,1,6,0,0,0,0
109,E-Motion,3,1,1,6,1,0,0,0
130,ETA,3,1,1,6,3,0,0,0
299,S.T.C.,4,1,1,4,0,0,0,0


In [33]:
def check_for_superlow_winners(G):
    """
    Checks whether any procurement-winning companies still have 'SuperLow' match_confidence.
    Returns a list of such nodes and prints a summary.
    """
    superlow_nodes = []

    for node, data in G.nodes(data=True):
        if data.get("type") == "Company" and data.get("bid_winner") == "True":
            if data.get("match_confidence") == "SuperLow":
                superlow_nodes.append((node, data.get("name", "Unknown")))

    print(f"❗ Found {len(superlow_nodes)} procurement winners with 'SuperLow' confidence.")
    if superlow_nodes:
        print("🔍 Sample:")
        for node, name in superlow_nodes[:10]:
            print(f"  - Node ID: {node}, Name: {name}")

    return superlow_nodes

In [34]:
check_for_superlow_winners(Gf_clean)

❗ Found 0 procurement winners with 'SuperLow' confidence.


[]

In [39]:
sanctions = pd.read_csv('/Users/wiktorrajca/Desktop/Research/URAP_Fedyk/data/Black_Lists/Black_Lists_Data_Sets/open_sanctions.csv')

/var/folders/r2/9vnxdxyd26380tms_8k538b40000gn/T/ipykernel_27921/1351478014.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  sanctions = pd.read_csv('/Users/wiktorrajca/Desktop/Research/URAP_Fedyk/data/Black_Lists/Black_Lists_Data_Sets/open_sanctions.csv')


In [ ]:
sanctions.groupby('name').count()

,id,schema,aliases,birth_date,countries,addresses,identifiers,sanctions,phones,emails,dataset,first_seen,last_seen,last_change
name,,,,,,,,,,,,,,
""" KLJUČAVNIČARSTVO "",SERVISIRANJE IN MONTAŽA MIRAN VERTAČNIK S.P.",1,1,0,0,1,0,1,0,0,0,1,1,1,1
"""ABU AL-HASAN AL MADANI""",2,2,1,0,0,0,0,2,0,0,2,2,2,2
"""AGRICOLA RUELAS"", SOCIEDAD DE PRODUCCION RURAL DE RESPONSABILIDAD ILIMITADA",1,1,1,0,0,0,1,1,0,0,1,1,1,1
"""AHMED THE TANZANIAN""",1,1,1,0,0,0,0,1,0,0,1,1,1,1
"""ALONDRA PRODUCE"", SOCIEDAD DE PRODUCCION RURAL DE RESPONSABILIDAD ILIMITADA",1,1,1,0,0,0,1,1,0,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Željko Milovanović, s.p., poslovno svetovanje in druge storitve",1,1,0,0,1,0,1,0,0,0,1,1,1,1
“AGIDEL-AS” LLP,2,2,0,0,2,1,0,2,0,0,2,2,2,2
“Interyer Insaat” Mahdud Masuliyyatli Cəmiyyati,1,1,1,0,1,1,0,1,0,0,1,1,1,1


In [61]:
sanctions[sanctions['id']=='NK-Gbik6Hp3G8o9nPkMzyq68M']

,id,schema,name,aliases,birth_date,countries,addresses,identifiers,sanctions,phones,emails,dataset,first_seen,last_seen,last_change
8280,NK-Gbik6Hp3G8o9nPkMzyq68M,LegalEntity,ANTARES LLC,ANTARES OOO;LIMITED LIABILITY COMPANY ANTARES,NaN,ru,"UL. SMIRNOVSKAYA D. 10, STR. 8. KABINET 10, MO...",C1UFLB6D9TE5;N744BK9A7714;NAPRCFCSM743,Reciprocal - 2022-02-22,NaN,NaN,US SAM Procurement Exclusions,2024-06-14T06:55:20,2025-01-07T07:55:02,2024-12-03T11:56:57


In [62]:
sanctions

,id,schema,name,aliases,birth_date,countries,addresses,identifiers,sanctions,phones,emails,dataset,first_seen,last_seen,last_change
0,NK-22HtK7WrxZ2sU3rmhz6PuZ,Person,MICHAEL KUAJIEN,MICHAEL KUAJIAN;MICHAEL KUAJIEN DUER MAYOK,NaN,ke,"NAIROBI, KEN",NaN,Reciprocal - 2019-12-10,NaN,NaN,US SAM Procurement Exclusions,2024-06-14T06:55:20,2025-01-07T07:55:02,2024-09-10T07:55:01
1,NK-22JRrAHhpxzN8wN5pP6EGo,Person,YASIN M HUSSAIN,Yasin M. Hussain,1965-01-19,us,"11646 SUN CIRCLE WAY, COLUMBIA, MD 21044;11646...",NaN,1996-06-10;US HHS OIG List of Excluded Individ...,NaN,NaN,US Health and Human Sciences Inspector General...,2024-05-08T21:57:02,2025-01-07T07:43:04,2024-10-15T07:43:06
2,NK-22MHXvQQufBgTjUWgUbWb8,LegalEntity,LIMITED LIABILITY COMPANY SPECIALIZED DEVELOPE...,LLC SPECIALIZED DEVELOPER ALABUGA SOUTH PARK,NaN,ru,"UL. SH-2 (OEZ ALABUGA TER.), D. 15/5, POMESHCH...",C526RHBSEVC3;U1EZWUPPYNN3,Reciprocal - 2024-02-23,NaN,NaN,US SAM Procurement Exclusions,2024-06-14T06:55:20,2025-01-07T07:55:02,2024-12-03T11:56:57
3,NK-22T6dTsc6umYgEA33vME3t,LegalEntity,DIGITAL MARKETING AWARDS FZ LLC,SWISS DRAGONS MANAGEMENT DIGITAL MARKETING AWARDS,NaN,ae,"COMPASS BUILDING FDRK 2508, AL SHOHADA ROAD, A...",EVSNP8SMGZF5;PCEXVGTXWFB4,Reciprocal - 2024-08-23,NaN,NaN,US SAM Procurement Exclusions,2024-09-21T07:55:17,2025-01-07T07:55:02,2024-12-03T11:56:57
4,NK-22cQD3a8GDRSuoW5mus3F3,LegalEntity,EKVIK LIMITED LIABILITY COMPANY,NaN,NaN,ru,"PER. DUKHOVSKOI D. 17, KORPUS 1, FLOOR 1 PODYE...",Y6MSPLJPJ9J5,Reciprocal - 2024-10-30,NaN,NaN,US SAM Procurement Exclusions,2024-11-01T07:55:06,2025-01-07T07:55:02,2024-12-03T11:56:57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205983,wbdeb-888293,LegalEntity,HEBEI CONSTRUCTION GROUP CORPORATION LIMITED Q...,NaN,NaN,cn,"NO. 382,WEST DEVELOPMENT ROAD, QINGYUAN DISTRI...",NaN,Cross Debarment: ADB - 2024-08-22 - 2027-07-22,NaN,NaN,WorldBank Debarred Providers,2024-08-23T15:03:03,2025-01-07T15:03:01,2024-11-28T15:03:03
205984,wbdeb-888338,LegalEntity,HEBEI CONSTRUCTION GROUP CORPORATION LIMITED M...,NaN,NaN,cn,"NO 859, XIANGYANG SOUTH STREET, JINGXIU DISTRI...",NaN,Cross Debarment: ADB - 2024-08-22 - 2027-07-22,NaN,NaN,WorldBank Debarred Providers,2024-08-23T15:03:03,2025-01-07T15:03:01,2024-11-28T15:03:03
205985,wbdeb-895566,LegalEntity,SINO-KENYA ENGINEERING GROUP COMNPANY LIMITED,NaN,NaN,ke,"NEXTGEN APARTMENTS, BLOCK B, HOUSE NO. 41, MOM...",NaN,Cross Debarment: AfDB - 2021-10-18 - 2999-12-31,NaN,NaN,WorldBank Debarred Providers,2024-09-17T15:03:04,2025-01-07T15:03:01,2024-11-28T15:03:03
205986,wbdeb-927832,LegalEntity,MR. RUFAT MAJIDOV,NaN,NaN,az,"BINAGADI HIGHWAY, MADAN STREET 4 AZ1053, BAKU,...",NaN,Obstructive Practices - 2024-12-10 - 2027-08-09,NaN,NaN,WorldBank Debarred Providers,2024-12-11T15:03:02,2025-01-07T15:03:01,2024-12-11T15:03:02


In [67]:
# Group by 'name' and count all columns
grouped_counts = sanctions.groupby('name').count()

# Filter to keep only rows where the count of 'id' is greater than 1
result = grouped_counts[grouped_counts['id'] > 4]
result

,id,schema,aliases,birth_date,countries,addresses,identifiers,sanctions,phones,emails,dataset,first_seen,last_seen,last_change
name,,,,,,,,,,,,,,
AL-AQSA ASBL,8,8,8,0,0,0,8,8,0,0,8,8,8,8
AL-AQSA CHARITABLE FOUNDATION,9,9,9,0,0,0,9,9,0,0,9,9,9,9
AL-AQSA CHARITABLE ORGANIZATION,9,9,9,0,0,0,9,9,0,0,9,9,9,9
AL-AQSA E.V.,9,9,9,0,1,1,9,9,0,0,9,9,9,9
AL-AQSA FOUNDATION,9,9,9,0,3,3,9,9,0,0,9,9,9,9
AL-AQSA INTERNATIONAL FOUNDATION,9,9,9,0,0,0,9,9,0,0,9,9,9,9
AL-AQSA ISLAMIC CHARITABLE SOCIETY,9,9,9,0,0,0,9,9,0,0,9,9,9,9
AL-AQSA SINABIL ESTABLISHMENT,9,9,9,0,0,0,9,9,0,0,9,9,9,9
AL-AQSA SPANM I STIFTELSE,9,9,9,0,0,0,9,9,0,0,9,9,9,9


In [3]:
ted = pd.read_csv("/Users/wiktorrajca/Desktop/Research/URAP_Fedyk/export_CAN_2019.csv")

/var/folders/r2/9vnxdxyd26380tms_8k538b40000gn/T/ipykernel_1517/3393530238.py:1: DtypeWarning: Columns (18,20) have mixed types. Specify dtype option on import or set low_memory=False.
  ted = pd.read_csv("/Users/wiktorrajca/Desktop/Research/URAP_Fedyk/export_CAN_2019.csv")


In [6]:
ted["WIN_NATIONALID"].isna().sum()

np.int64(713147)

In [9]:
ted["WIN_NATIONALID"].isna().sum()/ted.shape[0]

np.float64(0.7197608827516045)

In [12]:
ted["WIN_COUNTRY_CODE"].isna().sum()/ted.shape[0]

np.float64(0.31477345326202477)

In [16]:
sanctions_merged = pd.read_csv("/Users/wiktorrajca/Documents/GitHub/Data-Science-Honors-Thesis/code/output_test/Sanctions/merged_result_3.csv")

In [ ]:
sanctions_merged[sanctions_merged['name'] == 'ANTARES LLC']

,id,schema,name,aliases,birth_date,countries,addresses,identifiers,sanctions,phones,emails,dataset,first_seen,last_seen,last_change,name.1,bvdidnumber
73,NK-Gbik6Hp3G8o9nPkMzyq68M,LegalEntity,ANTARES LLC,ANTARES OOO;LIMITED LIABILITY COMPANY ANTARES,NaN,ru,"UL. SMIRNOVSKAYA D. 10, STR. 8. KABINET 10, MO...",C1UFLB6D9TE5;N744BK9A7714;NAPRCFCSM743,Reciprocal - 2022-02-22,NaN,NaN,US SAM Procurement Exclusions,2024-06-14T06:55:20,2025-01-07T07:55:02,2024-12-03T11:56:57,ANTARES LLC,UA23792943
74,NK-Gbik6Hp3G8o9nPkMzyq68M,LegalEntity,ANTARES LLC,ANTARES OOO;LIMITED LIABILITY COMPANY ANTARES,NaN,ru,"UL. SMIRNOVSKAYA D. 10, STR. 8. KABINET 10, MO...",C1UFLB6D9TE5;N744BK9A7714;NAPRCFCSM743,Reciprocal - 2022-02-22,NaN,NaN,US SAM Procurement Exclusions,2024-06-14T06:55:20,2025-01-07T07:55:02,2024-12-03T11:56:57,ANTARES LLC,UA32033608
75,NK-Gbik6Hp3G8o9nPkMzyq68M,LegalEntity,ANTARES LLC,ANTARES OOO;LIMITED LIABILITY COMPANY ANTARES,NaN,ru,"UL. SMIRNOVSKAYA D. 10, STR. 8. KABINET 10, MO...",C1UFLB6D9TE5;N744BK9A7714;NAPRCFCSM743,Reciprocal - 2022-02-22,NaN,NaN,US SAM Procurement Exclusions,2024-06-14T06:55:20,2025-01-07T07:55:02,2024-12-03T11:56:57,ANTARES LLC,UA25107408
76,NK-Gbik6Hp3G8o9nPkMzyq68M,LegalEntity,ANTARES LLC,ANTARES OOO;LIMITED LIABILITY COMPANY ANTARES,NaN,ru,"UL. SMIRNOVSKAYA D. 10, STR. 8. KABINET 10, MO...",C1UFLB6D9TE5;N744BK9A7714;NAPRCFCSM743,Reciprocal - 2022-02-22,NaN,NaN,US SAM Procurement Exclusions,2024-06-14T06:55:20,2025-01-07T07:55:02,2024-12-03T11:56:57,ANTARES LLC,UA30145152
77,NK-Gbik6Hp3G8o9nPkMzyq68M,LegalEntity,ANTARES LLC,ANTARES OOO;LIMITED LIABILITY COMPANY ANTARES,NaN,ru,"UL. SMIRNOVSKAYA D. 10, STR. 8. KABINET 10, MO...",C1UFLB6D9TE5;N744BK9A7714;NAPRCFCSM743,Reciprocal - 2022-02-22,NaN,NaN,US SAM Procurement Exclusions,2024-06-14T06:55:20,2025-01-07T07:55:02,2024-12-03T11:56:57,ANTARES LLC,UA258063607
78,NK-Gbik6Hp3G8o9nPkMzyq68M,LegalEntity,ANTARES LLC,ANTARES OOO;LIMITED LIABILITY COMPANY ANTARES,NaN,ru,"UL. SMIRNOVSKAYA D. 10, STR. 8. KABINET 10, MO...",C1UFLB6D9TE5;N744BK9A7714;NAPRCFCSM743,Reciprocal - 2022-02-22,NaN,NaN,US SAM Procurement Exclusions,2024-06-14T06:55:20,2025-01-07T07:55:02,2024-12-03T11:56:57,ANTARES LLC,UA25342109
79,NK-Gbik6Hp3G8o9nPkMzyq68M,LegalEntity,ANTARES LLC,ANTARES OOO;LIMITED LIABILITY COMPANY ANTARES,NaN,ru,"UL. SMIRNOVSKAYA D. 10, STR. 8. KABINET 10, MO...",C1UFLB6D9TE5;N744BK9A7714;NAPRCFCSM743,Reciprocal - 2022-02-22,NaN,NaN,US SAM Procurement Exclusions,2024-06-14T06:55:20,2025-01-07T07:55:02,2024-12-03T11:56:57,ANTARES LLC,UA25342010
80,NK-Gbik6Hp3G8o9nPkMzyq68M,LegalEntity,ANTARES LLC,ANTARES OOO;LIMITED LIABILITY COMPANY ANTARES,NaN,ru,"UL. SMIRNOVSKAYA D. 10, STR. 8. KABINET 10, MO...",C1UFLB6D9TE5;N744BK9A7714;NAPRCFCSM743,Reciprocal - 2022-02-22,NaN,NaN,US SAM Procurement Exclusions,2024-06-14T06:55:20,2025-01-07T07:55:02,2024-12-03T11:56:57,ANTARES LLC,UA25418166
81,NK-Gbik6Hp3G8o9nPkMzyq68M,LegalEntity,ANTARES LLC,ANTARES OOO;LIMITED LIABILITY COMPANY ANTARES,NaN,ru,"UL. SMIRNOVSKAYA D. 10, STR. 8. KABINET 10, MO...",C1UFLB6D9TE5;N744BK9A7714;NAPRCFCSM743,Reciprocal - 2022-02-22,NaN,NaN,US SAM Procurement Exclusions,2024-06-14T06:55:20,2025-01-07T07:55:02,2024-12-03T11:56:57,ANTARES LLC,UA25421107
82,NK-Gbik6Hp3G8o9nPkMzyq68M,LegalEntity,ANTARES LLC,ANTARES OOO;LIMITED LIABILITY COMPANY ANTARES,NaN,ru,"UL. SMIRNOVSKAYA D. 10, STR. 8. KABINET 10, MO...",C1UFLB6D9TE5;N744BK9A7714;NAPRCFCSM743,Reciprocal - 2022-02-22,NaN,NaN,US SAM Procurement Exclusions,2024-06-14T06:55:20,2025-01-07T07:55:02,2024-12-03T11:56:57,ANTARES LLC,UA22454210


In [21]:
ted["WIN_NAME"].str.lower.str.contains('antares')

AttributeError: 'function' object has no attribute 'str'

In [29]:
ted[ted["WIN_NAME"].str.lower().str.contains('antares', na=False)][['WIN_NAME', 'WIN_COUNTRY_CODE', 'WIN_TOWN']]

,WIN_NAME,WIN_COUNTRY_CODE,WIN_TOWN
32829,Antares IT,FR,Levallois-Perret
34087,Antares TP,FR,Baie-Mahault
381102,Antares România S.R.L.---Ovo Design Furniture ...,RO---RO---RO,Cluj-Napoca---București---București
410681,"Antares Consulting — Consultoria de Gestão, Lda.",PT,Lisboa
488364,Antares IT,FR,Levallois Perret
488367,Antares IT,FR,Levallois Perret
575617,Antares IT,FR,Levallois-Perret
730768,Antares IT,FR,Levallois-Perret
831237,Axis Specialty Ltd---Central Reinsurance Corpo...,BM---TW---CH---UK---UK---UK---DE---SE---JP---SI,"92 Pitts Bay Road, Pembroke, HM 08 Bermuda---1..."


In [30]:
sanctions_merged

,id,schema,name,aliases,birth_date,countries,addresses,identifiers,sanctions,phones,emails,dataset,first_seen,last_seen,last_change,name.1,bvdidnumber
0,NK-2CdXhyCy9XzyZgcFcHXrTZ,LegalEntity,LLC HAVERIM,CHAVERIM;CHAVERIM LLC HAVERIM;LIMITED LIABILIT...,NaN,ru,"NAB. OKTYABRSKAYA D. 104, K. 1 LIT. P, OFFICE ...",EVFWEDMZY2C5;EVG4X9K8WUF4;LEY4MWR5LB45,Reciprocal - 2024-08-23,NaN,NaN,US SAM Procurement Exclusions,2024-09-20T07:55:13,2025-01-07T07:55:02,2024-12-03T11:56:57,LLC HAVERIM,UA23839739
1,NK-2DoNAuynq6QjpaKV4CqXX9,LegalEntity,LLC INTERRESURS,NaN,NaN,ru,"UL. MARSHALA GOVOROVA D. 35, KORPUS 4 LIT. I, ...",XZ52FKLQAEL9,Reciprocal - 2024-10-30,NaN,NaN,US SAM Procurement Exclusions,2024-11-01T07:55:06,2025-01-07T07:55:02,2024-12-03T11:56:57,LLC INTERRESURS,UA25865927
2,NK-2DoNAuynq6QjpaKV4CqXX9,LegalEntity,LLC INTERRESURS,NaN,NaN,ru,"UL. MARSHALA GOVOROVA D. 35, KORPUS 4 LIT. I, ...",XZ52FKLQAEL9,Reciprocal - 2024-10-30,NaN,NaN,US SAM Procurement Exclusions,2024-11-01T07:55:06,2025-01-07T07:55:02,2024-12-03T11:56:57,LLC INTERRESURS,UA25760887
3,NK-2DoNAuynq6QjpaKV4CqXX9,LegalEntity,LLC INTERRESURS,NaN,NaN,ru,"UL. MARSHALA GOVOROVA D. 35, KORPUS 4 LIT. I, ...",XZ52FKLQAEL9,Reciprocal - 2024-10-30,NaN,NaN,US SAM Procurement Exclusions,2024-11-01T07:55:06,2025-01-07T07:55:02,2024-12-03T11:56:57,LLC INTERRESURS,UA25760858
4,NK-2DoNAuynq6QjpaKV4CqXX9,LegalEntity,LLC INTERRESURS,NaN,NaN,ru,"UL. MARSHALA GOVOROVA D. 35, KORPUS 4 LIT. I, ...",XZ52FKLQAEL9,Reciprocal - 2024-10-30,NaN,NaN,US SAM Procurement Exclusions,2024-11-01T07:55:06,2025-01-07T07:55:02,2024-12-03T11:56:57,LLC INTERRESURS,UA25257852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2365,usgsa-s4mrtyf18,LegalEntity,ANDA CO.,ANDA COMPANY,NaN,sa,NaN,QQW2UKR4MVY5,Reciprocal - 2022-05-24,NaN,NaN,US SAM Procurement Exclusions,2024-11-02T07:55:02,2025-01-07T07:55:02,2024-12-03T11:56:57,ANDA CO.,UA24130977
2366,usgsa-s4mrtyf18,LegalEntity,ANDA CO.,ANDA COMPANY,NaN,sa,NaN,QQW2UKR4MVY5,Reciprocal - 2022-05-24,NaN,NaN,US SAM Procurement Exclusions,2024-11-02T07:55:02,2025-01-07T07:55:02,2024-12-03T11:56:57,ANDA CO.,UA30297422
2367,usgsa-s4mrtyf19,LegalEntity,ANDA LTD.,ANDA COMPANY,NaN,sa,NaN,QQZ4GBGJE7N4,Reciprocal - 2022-05-24,NaN,NaN,US SAM Procurement Exclusions,2024-11-02T07:55:02,2025-01-07T07:55:02,2024-12-03T11:56:57,ANDA LTD.,UA24130977
2368,usgsa-s4mrtyf19,LegalEntity,ANDA LTD.,ANDA COMPANY,NaN,sa,NaN,QQZ4GBGJE7N4,Reciprocal - 2022-05-24,NaN,NaN,US SAM Procurement Exclusions,2024-11-02T07:55:02,2025-01-07T07:55:02,2024-12-03T11:56:57,ANDA LTD.,UA30297422


#### REMOVING FLAGGED ENTITIES

In [31]:
import networkx as nx
import os

def remove_flagged_entities(G, output_path="cleaned_graph_without_flags.graphml"):
    """
    Removes all flagged entity markings from the graph:
    - Deletes 'Flagged' type nodes
    - Deletes edges of type 'FLAGGED_LINK'
    - Removes 'flagged' and 'flagged_reason' attributes from remaining nodes
    Saves the cleaned graph to a new GraphML file.

    Parameters:
        G (networkx.Graph): The input graph.
        output_path (str): Path to save the cleaned graph.

    Returns:
        networkx.Graph: The cleaned graph.
    """
    G_cleaned = G.copy()

    # 1. Remove edges labeled 'FLAGGED_LINK'
    flagged_edges = [(u, v) for u, v, d in G_cleaned.edges(data=True)
                     if d.get("relationship") == "FLAGGED_LINK"]
    G_cleaned.remove_edges_from(flagged_edges)

    # 2. Remove all nodes of type 'Flagged'
    flagged_nodes = [n for n, d in G_cleaned.nodes(data=True)
                     if d.get("type") == "Flagged"]
    G_cleaned.remove_nodes_from(flagged_nodes)

    # 3. Clean attributes from companies that were marked flagged
    for node, data in G_cleaned.nodes(data=True):
        if "flagged" in data:
            del data["flagged"]
        if "flagged_reason" in data:
            del data["flagged_reason"]

    # 4. Save graph
    nx.write_graphml(G_cleaned, output_path)
    print(f"✅ Cleaned graph saved to {output_path}")
    return G_cleaned

In [33]:
france = nx.read_graphml(f"procurement_graph_FR_clean.graphml")

In [34]:
france_no_flag = remove_flagged_entities(france)

✅ Cleaned graph saved to cleaned_graph_without_flags.graphml


In [39]:
summarize_graph(france_no_flag)


===== Graph Summary =====
Total Nodes: 606726
Total Edges: 431001

Node Types:
  - Procurement: 270132
  - Company: 336594

Edge Types:
  - WON: 270132
  - CONTROLS: 12973
  - OWNS: 147819
  - SUBSIDIARY_OF: 77

Flagged Companies: 0



#### WORK ON STATISTICS AND ROBOUSTNESS

In [ ]:
import pandas as pd
import networkx as nx
from collections import defaultdict
from tqdm import tqdm

def compute_expected_flagged_matches(flagged_df):
    """
    Compute expected number of flagged entity matches per name.
    """
    expected_matches = (
        flagged_df[flagged_df['schema'] == 'LegalEntity']
        .groupby('name')['id']
        .nunique()
        .to_dict()
    )
    country_by_name = (
        flagged_df[flagged_df['schema'] == 'LegalEntity']
        .dropna(subset=['countries'])
        .groupby('name')['countries']
        .first()
        .to_dict()
    )
    return expected_matches, country_by_name

def determine_flagged_robustness_level(node, G, expected_matches, country_by_name):
    """
    Determines the robustness level (1 to 3) for a flagged node.
    """
    name = G.nodes[node].get("name")
    country = G.nodes[node].get("country", "").upper()
    match_count = sum(
        1 for n, d in G.nodes(data=True)
        if d.get("flagged") == "True" and d.get("name") == name
    )
    expected = expected_matches.get(name, 1)
    correct_country = country_by_name.get(name, "").upper()
    
    if match_count <= expected and country == correct_country:
        return 3
    elif match_count <= expected:
        return 2
    else:
        return 1

def filter_graph_by_match_confidence(G, min_confidence="Very High"):
    """
    Returns a subgraph of G with nodes that have at least the given match confidence.
    """
    confidence_levels = ["SuperLow", "Low", "Medium", "High", "Very High"]
    keep_levels = confidence_levels[confidence_levels.index(min_confidence):]
    
    keep_nodes = [
        n for n, d in G.nodes(data=True)
        if d.get("type") != "Company" or d.get("match_confidence") in keep_levels
    ]
    return G.subgraph(keep_nodes).copy()

def compute_flagged_statistics(G, flagged_df):
    """
    Computes flagged entity stats across 15 robustness combinations.
    """
    expected_matches, country_by_name = compute_expected_flagged_matches(flagged_df)

    # Match confidence tiers
    match_conf_tiers = ["All", "No Low", "No Low/Medium", "High+", "Very High"]
    # Sanction match levels
    sanction_robust_levels = [1, 2, 3]

    summary = []

    for conf in match_conf_tiers:
        if conf == "All":
            G_sub = G.copy()
        else:
            min_conf = {
                "No Low": "Medium",
                "No Low/Medium": "High",
                "High+": "High",
                "Very High": "Very High"
            }[conf]
            G_sub = filter_graph_by_match_confidence(G, min_conf)

        for level in sanction_robust_levels:
            flagged_nodes = [
                n for n, d in G_sub.nodes(data=True)
                if d.get("flagged") == "True"
            ]

            if level > 1:
                filtered_flagged = []
                for n in flagged_nodes:
                    lvl = determine_flagged_robustness_level(n, G_sub, expected_matches, country_by_name)
                    if lvl >= level:
                        filtered_flagged.append(n)
                flagged_nodes = filtered_flagged

            winners = {
                n for n, d in G_sub.nodes(data=True)
                if d.get("type") == "Company" and d.get("bid_winner") == "True"
            }

            # Count direct flagged winners
            direct_flagged_winners = len(set(flagged_nodes).intersection(winners))

            # Count shortest path links
            linked_to_winners = 0
            for flagged in flagged_nodes:
                for winner in winners:
                    if nx.has_path(G_sub, flagged, winner) or nx.has_path(G_sub, winner, flagged):
                        linked_to_winners += 1
                        break

            summary.append({
                "Match_Confidence": conf,
                "Sanction_Robustness": level,
                "Flagged_Entities": len(flagged_nodes),
                "Direct_Winners": direct_flagged_winners,
                "Linked_to_Winners": linked_to_winners
            })

    return pd.DataFrame(summary)


In [66]:
compute_flagged_statistics(france, sanc)

KeyboardInterrupt: 

In [71]:
import pandas as pd
import networkx as nx
from collections import defaultdict
from tqdm import tqdm

def compute_flagged_statistics_with_logging(G, sanctions_df):
    """
    Compute statistics of how flagged nodes (companies or individuals) connect to procurement winners
    across different robustness levels.

    Parameters:
        G (nx.DiGraph): Cleaned graph.
        sanctions_df (pd.DataFrame): Full sanctions dataset (pre-matching).

    Returns:
        pd.DataFrame: Summary statistics across 15 robustness combinations.
    """

    print("🔍 Computing expected matches for flagged names...")
    # 1. Estimate expected match counts and countries for each name from full sanctions dataset
    expected_match_counts = sanctions_df.groupby("name")["id"].nunique().to_dict()
    expected_country_codes = sanctions_df.groupby("name")["countries"].apply(
        lambda x: set([code for sublist in x.dropna().str.split(",") for code in sublist])
    ).to_dict()

    print("🔍 Extracting flagged and procurement nodes...")
    # 2. Extract flagged and procurement-winning nodes
    flagged_nodes = {
        node: data for node, data in G.nodes(data=True)
        if data.get("type") == "Flagged" or data.get("flagged") == "True"
    }
    winner_nodes = {
        node for node, data in G.nodes(data=True)
        if data.get("type") == "Company" and data.get("bid_winner") == "True"
    }

    print("🔁 Precomputing paths up to length 3 from flagged to winner nodes...")
    # 3. Precompute shortest paths up to length 3
    shortest_paths = {}
    for f in tqdm(flagged_nodes, desc="Shortest paths"):
        shortest_paths[f] = {}
        try:
            paths = nx.single_source_shortest_path_length(G, f, cutoff=3)
            for target, dist in paths.items():
                if target in winner_nodes:
                    shortest_paths[f][target] = dist
        except nx.NetworkXError:
            continue

    print("⚙️ Defining robustness levels...")
    # 4. Define robustness levels
    match_filters = {
        "All": lambda d: True,
        "No Low": lambda d: d.get("match_confidence") not in {"Low"},
        "No Medium": lambda d: d.get("match_confidence") not in {"Low", "Medium"},
        "High+": lambda d: d.get("match_confidence") in {"High", "Very High"},
        "Very High": lambda d: d.get("match_confidence") == "Very High",
    }

    sanction_filters = {
        "All": lambda name, node: True,
        "Respect Expected": lambda name, node: True,  # limited by expected counts
        "Respect Expected + Country": lambda name, node: (
            node.get("country") in expected_country_codes.get(name, set())
        )
    }

    print("📊 Computing statistics for 15 robustness combinations...")
    rows = []

    for m_label, m_filter in match_filters.items():
        for s_label, s_filter in sanction_filters.items():
            print(f"🔍 Running {m_label} | {s_label}")
            filtered_flagged = []
            grouped = defaultdict(list)

            for node, data in flagged_nodes.items():
                name = data.get("name", "Unknown")
                if not m_filter(data):
                    continue
                if not s_filter(name, data):
                    continue
                grouped[name].append((node, data))

            # Apply expected match limit
            if s_label in {"Respect Expected", "Respect Expected + Country"}:
                for name, entries in grouped.items():
                    limit = expected_match_counts.get(name, len(entries))
                    filtered_flagged.extend([x for x in entries[:limit]])
            else:
                filtered_flagged = [x for lst in grouped.values() for x in lst]

            winner_flagged = 0
            dist_1, dist_2, dist_3plus = 0, 0, 0

            for node, _ in filtered_flagged:
                if node in winner_nodes:
                    winner_flagged += 1
                elif node in shortest_paths:
                    min_dist = min(shortest_paths[node].values(), default=None)
                    if min_dist == 1:
                        dist_1 += 1
                    elif min_dist == 2:
                        dist_2 += 1
                    elif min_dist == 3:
                        dist_3plus += 1

            rows.append({
                "Match_Level": m_label,
                "Sanction_Level": s_label,
                "Total_Flagged_Considered": len(filtered_flagged),
                "Flagged_Winners": winner_flagged,
                "Distance_1": dist_1,
                "Distance_2": dist_2,
                "Distance_3+": dist_3plus
            })

    result_df = pd.DataFrame(rows)
    print("✅ Statistics computation complete.")
    return result_df

In [72]:
results_france = compute_flagged_statistics_with_logging(france, sanc)

🔍 Computing expected matches for flagged names...
🔍 Extracting flagged and procurement nodes...
🔁 Precomputing paths up to length 3 from flagged to winner nodes...


Shortest paths: 100%|██████████| 663/663 [00:00<00:00, 38481.45it/s]

⚙️ Defining robustness levels...
📊 Computing statistics for 15 robustness combinations...
🔍 Running All | All
🔍 Running All | Respect Expected
🔍 Running All | Respect Expected + Country
🔍 Running No Low | All
🔍 Running No Low | Respect Expected
🔍 Running No Low | Respect Expected + Country
🔍 Running No Medium | All
🔍 Running No Medium | Respect Expected
🔍 Running No Medium | Respect Expected + Country
🔍 Running High+ | All
🔍 Running High+ | Respect Expected
🔍 Running High+ | Respect Expected + Country
🔍 Running Very High | All
🔍 Running Very High | Respect Expected
🔍 Running Very High | Respect Expected + Country
✅ Statistics computation complete.


In [73]:
results_france

,Match_Level,Sanction_Level,Total_Flagged_Considered,Flagged_Winners,Distance_1,Distance_2,Distance_3+
0,All,All,663,654,4,0,0
1,All,Respect Expected,591,582,4,0,0
2,All,Respect Expected + Country,0,0,0,0,0
3,No Low,All,376,367,4,0,0
4,No Low,Respect Expected,376,367,4,0,0
5,No Low,Respect Expected + Country,0,0,0,0,0
6,No Medium,All,26,17,4,0,0
7,No Medium,Respect Expected,26,17,4,0,0
8,No Medium,Respect Expected + Country,0,0,0,0,0
9,High+,All,17,17,0,0,0


In [78]:
summarize_graph(france)


===== Graph Summary =====
Total Nodes: 606730
Total Edges: 431038

Node Types:
  - Procurement: 270132
  - Company: 336594
  - Flagged: 4

Edge Types:
  - WON: 270132
  - CONTROLS: 12973
  - OWNS: 147819
  - FLAGGED_LINK: 37
  - SUBSIDIARY_OF: 77

Flagged Companies: 659

